In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import warnings
import gc
from collections import defaultdict
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Ruta al archivo Excel
file_path = 'C:/Users/smend/OneDrive/Documents/Tesis/renta_CDMX_2024.csv'

# Leer el archivo 
df = pd.read_csv(file_path)

# Mostrar las primeras filas
df.head()

C:\Users\smend\AppData\Local\Temp\ipykernel_18284\3547094725.py:5: DtypeWarning: Columns (0,2,11,13,15,16,17,21,23) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


,direccion,id_tipo_propiedad,tipo_propiedad,area_construida,recamaras,banos,edad,estacionamientos,latitud,longitud,...,id_cve_geo_col,Colonia,precio_oferta,fecha_oferta,precio_m2,cp,superficie_terreno,amenidades,elevador,ubicacion
0,NaN,4.0,Departamento,74.0,3.0,2.0,5.0,1.0,19.464082,-99.120232,...,9005245.0,Guadalupe Tepeyac,"16,000",15/04/2024,216.216216,7840.0,74.0,NaN,NaN,POINT (-99.1202321 19.4640821)
1,NaN,4.0,Departamento,200.0,3.0,3.0,7.0,3.0,19.382607,-99.161064,...,90142394.0,Del Valle Centro,"55,000",21/02/2024,275.000000,3100.0,200.0,NaN,NaN,POINT (-99.1610642 19.3826071)
2,"Miguel Bernard, Col. La Escalera C.P. 07320, G...",4.0,Departamento,74.0,3.0,1.0,0.0,NaN,19.508087,-99.136730,...,9005217.0,Sierravista,"11,000",19/03/2024,148.648649,7320.0,NaN,"""{'amenidades': {'cocina_amueblada': None, 'pi...",0.0,POINT (-99.1367299 19.5080874)
3,"Norte 79 B 224 A - 1, Col. Un Hogar Para Cada ...",4.0,Departamento,45.0,1.0,1.0,0.0,NaN,19.475753,-99.175971,...,900221.0,Un Hogar Para Cada Trabajador,"7,600",21/03/2024,168.888889,2060.0,NaN,"""{'amenidades': {'cocina_amueblada': None, 'pi...",0.0,POINT (-99.1759713 19.4757532)
4,"SN, Col. Romero Rubio C.P. 15400, Venustiano C...",4.0,Departamento,65.0,2.0,1.0,0.0,NaN,19.442391,-99.096380,...,9017280.0,Romero Rubio,"11,000",16/03/2024,169.230769,15400.0,NaN,"""{'amenidades': {'cocina_amueblada': None, 'pi...",0.0,POINT (-99.09638 19.442391)


In [3]:
print(df.columns)

Index(['direccion', 'id_tipo_propiedad', 'tipo_propiedad', 'area_construida',
       'recamaras', 'banos', 'edad', 'estacionamientos', 'latitud', 'longitud',
       'id_cve_geo_edo', 'Estado', 'id_cve_geo_muni', 'Minicipio',
       'id_cve_geo_col', 'Colonia', ' precio_oferta ', 'fecha_oferta',
       'precio_m2', 'cp', 'superficie_terreno', 'amenidades', 'elevador',
       'ubicacion'],
      dtype='object')


In [4]:
df.columns = df.columns.str.strip() # Eliminar espacios al principio y al final de los nombres de columnas

# Eliminar columnas no deseadas
columnas_eliminar = [
    'direccion', 'id_tipo_propiedad', 'id_cve_geo_muni',
    'id_cve_geo_col', 'superficie_terreno', 'amenidades',
    'elevador', 'ubicacion'
]

df_clean = df.drop(columns=columnas_eliminar)

# Reordenar columnas 
nuevo_orden = [
    'tipo_propiedad', 'area_construida', 'recamaras', 'banos',
    'edad', 'estacionamientos', 'latitud', 'longitud', 'Municipio',
    'Colonia', 'precio_oferta', 'fecha_oferta', 'precio_m2', 'cp'
]

df_clean = df_clean.rename(columns={'Minicipio': 'Municipio'}) 

df_clean = df_clean[nuevo_orden]

In [5]:
df_clean.head()

,tipo_propiedad,area_construida,recamaras,banos,edad,estacionamientos,latitud,longitud,Municipio,Colonia,precio_oferta,fecha_oferta,precio_m2,cp
0,Departamento,74.0,3.0,2.0,5.0,1.0,19.464082,-99.120232,Gustavo A. Madero,Guadalupe Tepeyac,"16,000",15/04/2024,216.216216,7840.0
1,Departamento,200.0,3.0,3.0,7.0,3.0,19.382607,-99.161064,Benito Juárez,Del Valle Centro,"55,000",21/02/2024,275.000000,3100.0
2,Departamento,74.0,3.0,1.0,0.0,NaN,19.508087,-99.136730,Gustavo A. Madero,Sierravista,"11,000",19/03/2024,148.648649,7320.0
3,Departamento,45.0,1.0,1.0,0.0,NaN,19.475753,-99.175971,Azcapotzalco,Un Hogar Para Cada Trabajador,"7,600",21/03/2024,168.888889,2060.0
4,Departamento,65.0,2.0,1.0,0.0,NaN,19.442391,-99.096380,Venustiano Carranza,Romero Rubio,"11,000",16/03/2024,169.230769,15400.0


Análisis Exploratorio de Datos (EDA):

In [6]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 753652 entries, 0 to 753651
Data columns (total 14 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   tipo_propiedad    124516 non-null  object 
 1   area_construida   124516 non-null  float64
 2   recamaras         118743 non-null  float64
 3   banos             115639 non-null  float64
 4   edad              109180 non-null  float64
 5   estacionamientos  75941 non-null   float64
 6   latitud           124516 non-null  float64
 7   longitud          124516 non-null  float64
 8   Municipio         124516 non-null  object 
 9   Colonia           123469 non-null  object 
 10  precio_oferta     124516 non-null  object 
 11  fecha_oferta      124516 non-null  object 
 12  precio_m2         124516 non-null  float64
 13  cp                124515 non-null  float64
dtypes: float64(9), object(5)
memory usage: 80.5+ MB


In [7]:
df_CDMX = df_clean.dropna(subset=['recamaras', 'banos']) #Eliminar filas si un conjunto específico de columnas tiene nulos

In [8]:
df_CDMX.fillna(0, inplace=True)  # Reemplaza todos los nulos con 0

C:\Users\smend\AppData\Local\Temp\ipykernel_18284\3148866280.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_CDMX.fillna(0, inplace=True)  # Reemplaza todos los nulos con 0


In [9]:
#Convertir una columna de tipo object a category para ahorrar memoria o mejorar el rendimiento en operaciones analíticas


columnas_a_convertir = ['Municipio', 'Colonia', 'cp','tipo_propiedad']
for col in columnas_a_convertir:
    df_CDMX[col] = df_CDMX[col].astype('category')

C:\Users\smend\AppData\Local\Temp\ipykernel_18284\2184837273.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_CDMX[col] = df_CDMX[col].astype('category')
C:\Users\smend\AppData\Local\Temp\ipykernel_18284\2184837273.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_CDMX[col] = df_CDMX[col].astype('category')
C:\Users\smend\AppData\Local\Temp\ipykernel_18284\2184837273.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_in

In [10]:
df_CDMX = df_CDMX.copy()
df_CDMX['fecha_oferta'] = pd.to_datetime(df_CDMX['fecha_oferta'], dayfirst=True)

In [11]:
# Lista de las columnas a convertir
columnas_a_convertir = ['recamaras', 'banos', 'edad','estacionamientos']

# Convertir las columnas especificadas a enteros
df_CDMX[columnas_a_convertir] = df_CDMX[columnas_a_convertir].astype('int64')

In [12]:
df_CDMX.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 113835 entries, 0 to 124515
Data columns (total 14 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   tipo_propiedad    113835 non-null  category      
 1   area_construida   113835 non-null  float64       
 2   recamaras         113835 non-null  int64         
 3   banos             113835 non-null  int64         
 4   edad              113835 non-null  int64         
 5   estacionamientos  113835 non-null  int64         
 6   latitud           113835 non-null  float64       
 7   longitud          113835 non-null  float64       
 8   Municipio         113835 non-null  category      
 9   Colonia           113835 non-null  category      
 10  precio_oferta     113835 non-null  object        
 11  fecha_oferta      113835 non-null  datetime64[ns]
 12  precio_m2         113835 non-null  float64       
 13  cp                113835 non-null  category      
dtypes: c

In [13]:
df_CDMX.head()

,tipo_propiedad,area_construida,recamaras,banos,edad,estacionamientos,latitud,longitud,Municipio,Colonia,precio_oferta,fecha_oferta,precio_m2,cp
0,Departamento,74.0,3,2,5,1,19.464082,-99.120232,Gustavo A. Madero,Guadalupe Tepeyac,"16,000",2024-04-15,216.216216,7840.0
1,Departamento,200.0,3,3,7,3,19.382607,-99.161064,Benito Juárez,Del Valle Centro,"55,000",2024-02-21,275.000000,3100.0
2,Departamento,74.0,3,1,0,0,19.508087,-99.136730,Gustavo A. Madero,Sierravista,"11,000",2024-03-19,148.648649,7320.0
3,Departamento,45.0,1,1,0,0,19.475753,-99.175971,Azcapotzalco,Un Hogar Para Cada Trabajador,"7,600",2024-03-21,168.888889,2060.0
4,Departamento,65.0,2,1,0,0,19.442391,-99.096380,Venustiano Carranza,Romero Rubio,"11,000",2024-03-16,169.230769,15400.0
